# Fixing Weird Patterns When Plotting NEXRAD Level 3 Data

The motivation here comes from a thread on Twitter, indicating an issue when plotting NEXRAD Level 3 radial velocity (`NOU`).

Here is the radar image, plotted by David ([@dryglick](https://twitter.com/dryglick)):

<img src="../images/nexrad-plotting-issue.jpeg" alt="drawing" width="400"/>

Notice the odd pattern directly north/south and east/west of the radar...

There appears to be a crosshair of grey/darker pixels in our image.

This was identified as a [Moiré Pattern](https://en.wikipedia.org/wiki/Moiré_pattern), by Ryan May ([@dopplershift](https://twitter.com/dopplershift)), lead developer of MetPy, an open source python library for working with meteorological data.

## The Data
We are using [NEXRAD Level 3 Data](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00708), which is ground-based radar data in the United States, defined as:

"NEXRAD Level 3 products are used to remotely detect atmospheric features, such as precipitation, precipitation-type, storms, turbulence and wind, for operational forecasting and data research analysis."

This dataset is provided by the National Oceanic and Atmospheric Administration (NOAA).

### Data Access
We can access this dataset using the [NCEI NEXRAD Level 3 Search Portal](https://www.ncei.noaa.gov/access/search/data-search/weather-radar-level-iii). We **could** access data from Amazon Web Services (AWS) if we were interested in data **after 2020**, but since we are looking at at data from 2018, we need to use the NCEI archive.

David mentioned he was looking at velocity data (`N0U`) from Elgin Air Force Base in the Panhandle of Florida, with the station identifier of `KEVX`, at **1458 UTC on October 10, 2018**, which was hours before [Hurricane Michael](https://www.nhc.noaa.gov/data/tcr/AL142018_Michael.pdf) made landfall along the Gulf Coast.

We can enter the:
- Radar data field (`N0U`)
- Location (`KEVX`)
- Date/time (`10/10/2018 at 1458 UTC`)

into our query. Here is what it should lookl like on the webpage:

<img src="../images/nexrad-level3-radar-query.png" alt="radar-data-query" width="600"/>

Once we add this to our cart, and wait for the data to be sent to our email, we download via HTTP (through a website) or FTP. 

We can now unzip the compressed file, then take a look at our data file, reading in the `N0U` (`velocity`) field.

The specific file we are interested in is:

`KMOB_SDUS54_N0UEVX_201810101458`

## Recreate the Issue Using PyART

### Imports

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import pyart
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

### Read the data using PyART
When reading into PyART, we can use the [`pyart.io.read_level3`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_level3.html#pyart.io.read_nexrad_level3) module, **or**, we can use the [`pyart.io.read`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read.html#pyart.io.read) module which automatically detects the filetype being read in.

In [ ]:
radar = pyart.io.read("KMOB_SDUS54_N0UEVX_201810101458")

### Plot our Data Using `RadarMapDisplay`


#### Setting our Matplotlib style
Before we dig into plotting, let's set our background to dark since that is what the original image showed and it will be easier to identify the pattern in the data.

In [ ]:
plt.style.use("dark_background")

#### Plot our data without setting `alpha`

Since we want geographical boundaries on our map, let's use `RadarMapDisplay` with our `Radar` object to create the map.

When we create our plot, everything looks okay - we don't see the odd Moiré Pattern...

In [ ]:
fig = plt.figure(figsize=(10, 7))
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map(
    "velocity",
    sweep=0,
    vmin=-100,
    vmax=100,
    projection=ccrs.PlateCarree(),
    colorbar_label="radial velocity (m/s)",
    cmap="pyart_balance",
)
plt.xlim(-88, -83)
plt.ylim(28, 33)
plt.show()

#### Add in the `alpha` argument

When we add in the `alpha=0.8` argument, which adjusts the ***transparency*** of our field, we start to see the pattern. 

In [ ]:
fig = plt.figure(figsize=(10, 7))
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map(
    "velocity",
    alpha=0.8,
    sweep=0,
    vmin=-100,
    vmax=100,
    projection=ccrs.PlateCarree(),
    colorbar_label="radial velocity (m/s)",
    cmap="pyart_balance",
)

plt.xlim(-88, -83)
plt.ylim(28, 33)
plt.show()

## Dig into the Issue

### Investigate `plot_ppi_map`
Let's take a look at the specific plotting call being made with `plot_ppi_map` using `??`:

In [ ]:
??display.plot_ppi_map

We are using `matplotlib.pyplot.pcolormesh` (`plt.pcolormesh`) here to plot our radar data - using the included parameters:
- alpha (data transparency)
- vmin (minimum value to plot)
- vmax (maximum value to plot)
- cmap (colormap to use)
- norm (colormap normalization)
- transform (which projection to use)

For `pcolormesh` parameters **not included in this list**, `pyart` is using the default parameters...

### Investigate `pcolormesh` parameters
In a similar fashion to the `plot_ppi_map` function from `pyart`, we can take a look at the parameters for `plt.pcolormesh`:

In [ ]:
?plt.pcolormesh

#### Digging into `edgecolors`
Notice how one of the arguments included here is `edgecolors`, which by default, is `none`. One of the options here is to set these to be "face", which is the adjacent data color.

Let's test this out with our data to see if changing this parameter makes a difference...

### Try Modifying the `edgecolors` argument with `pcolormesh`
Let's test this out with our data!

First, let's read in the numpy array of data using `['data']`.

In [ ]:
velocity = radar.fields["velocity"]["data"]
velocity

Next, we need to access our coordinates in latitude/longitude form using `get_gate_lat_lon_alt(sweep)`

Since we only have a single sweep, we can set that to 0.

In [ ]:
lats, lons, alt = radar.get_gate_lat_lon_alt(0)
lats, lons

#### Plot our data with the default `edgecolors` argument
Notice how our pattern shows up again - we get the Moiré Pattern...

In [ ]:
plt.pcolormesh(
    lons,
    lats,
    velocity,
    alpha=0.8,
);

When we try passing in `edgecolors='face'` instead, it goes away!

In [ ]:
plt.pcolormesh(lons, lats, velocity, alpha=0.8, edgecolors="face");

## Plotting up our Solution
We found the issue! 😁 

Let's plot up the original figure, with features, colorbars, and all!

In [ ]:
# Setup our figure/geoaxis
fig = plt.figure(figsize=(10, 7))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

# Add state borders and the coastline
ax.add_feature(cfeature.STATES, edgecolor="white", linewidth=2)
ax.add_feature(cfeature.COASTLINE, edgecolor="white")

# Plot our velocity field using pcolormesh with edgecolors argument
mesh = ax.pcolormesh(
    lons, lats, velocity, vmin=-100, vmax=100, edgecolors="face", cmap="pyart_balance"
)

# Configure our colorbar
cbar = plt.colorbar(mesh)
cbar.ax.tick_params(labelsize=14)
cbar.set_label("Radial Velocity (m/s)", fontsize=16)

# Add gridlines
gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=2,
    color="gray",
    alpha=0.5,
    linestyle="--",
)

# Make sure labels are only plotted on the left and bottom
gl.xlabels_top = False
gl.ylabels_right = False

# Increase the fontsize of our gridline labels
gl.xlabel_style = {"fontsize": 14}
gl.ylabel_style = {"fontsize": 14}

# Add a title, using time information
plt.title(
    f"KEVX (Elgin Air Force Base) \n 0.5 Degree Radial Velocity \n {radar.time['units'][14:]}",
    fontsize=16,
)

plt.savefig("kevz_velocity_2018_10_10_1458.png", dpi=300)

## Fixing the Issue in PyART

Currently, within PyART, there is not a method of specificying the argument in `plot_ppi_map`, which is why we needed to add that custom section at the end using `plt.pcolormesh` directly.

We are submitting a pull request (PR) to change the default behavior here (`edgecolors='face'`).

**More importantly, this highlights a current limitation of our plotting API where we cannot specify matplotlib.pcolormesh arguments through `plot_ppi_map`. We will make sure this PR adds the ability to specify plotting arguements to pass to `plt.pcolormesh`**

## Conclusion

Within this post, we explored an odd pattern that shows up when setting `alpha` values when plotting radar data using `plot_ppi_map`(`pcolormesh`). We needed to change the `edgecolors` argument in `plt.pcolormesh` to `edgecolors='faces'`.

This also highlighted a need in the `plot_ppi_map` method in PyART to allow users to pass plotting arguments to `pcolormesh`.

We are appreciative of David for raising this question on Twitter, helping us to work together to find a solution and improve our plotting functionality in PyART!